## Naive Bayes

Naivni Bayesov klasifikator spada u algoritme **nagledanog mašinskog učenja**.

Naivni Bayes koristi probabilističku metodu, odnosno Bayesovu teoremu, koja opisuje verovatnoću nekog događaja na osnovu drugih događaja koji ga uslovljavaju.

Bayesova teorema se matematički iskazuje sledećom formulom:

$$
P(A|B) = \frac{P(B | A) \, P(A)}{P(B)}
$$

gde su:

* P(A) i P(B) verovatnoće događaja A i B (**nezavisni jedan od drugog** - ovo je važna pretpostavka Bayesove teoreme)
* P(A|B) je uslovna verovatnoća događaja A, ako se desio događaj B
* P(B|A) je verovatnoća B, ako se desio A 

#### Neformalno objašenjenje na jednostavnom primeru

Recimo da postoje dva događaja A i B:
* **A** - gledao sam film Terminator 2 (i da, to mi je omiljeni film)
* **B** - sedeo sam na kauču

Koja je verovatnoća da gledam Terminator 2, ako sedim na kauču?

![img/t2.jpg](img/t2.jpg)


Sad ova 2 događaja treba predstaviti kao Bayesove komponente.

Film Terminator 2 sam gledao dosta puta (stvarno) kad sam bio klinac. Bilo je dana kada sam ga gledao i po dva puta dnevno, zaredom. Ali, da uprostimo celu stvar, hajde da kažemo da sam ga gledao 50 puta za godinu dana, što znači:

```
P(A) = P(gledao sam Terminator 2) = 50 / 365 ~= 0.14, (14% verovatnoća da ću u danu odgledati Terminator 2)
```

Kada sam kod kuće, uglavnom sedim na kauču (u suprotnom sedim za računarom, gde drugde). Ako radim 8 sati dnevno i spavam 6 sati dnevno, znači da sam 10 sati dnevno kod kuće, od toga recimo 7 sati na kauču, dakle:


```
P(B) = P(sedeo sam na kauču) = 7 / 24 ~= 0.29, (29% verovatnoća da u toku dana sedim na kauču)
```

Film Terminator 2 uglavnom sam gledao kod kuće, recimo 95% od svih gledanja, i kada sam ga gledao kod kuće, retko sam ga gledao na računaru, možda 20% od svih gledanja, što dođe:

```
P(B|A) = P(sedeo sam na kauču dok gledam Terminator 2) = 0.95*(1-0.2) = 0.76
```

Ok, sada imamo sve što je potrebno da Bayesovom teoremom izračunamo koja je verovatnoća da sam gledao film Terminator 2 ako sedim na kauču:

```
P(A|B) = P(B|A)*P(A)/P(B) = (0.76 * 0.14) / 0.29 ~= 0.37
```

Ovo je već zabrinjavajuće... Ispalo je da, ako sam danas sedeo na kauču, postoji čak 37% šanse da sam gledao film Terminator 2. Ipak, ima smisla - dobar je film!

----


### Naivni Bayesov klasifikator
 
Naivni Bayesov klasifikator se veoma često koristi u problemu klasifikacije dokumenata. Naivni Bayes je dobar izbor za ovo jer je veoma brz i dobro se nosi sa velikim brojem osobina podataka (npr. reči u dokumentu), i zapravo je veoma efektivan. Naivni Bayes inače važi za jedne od najjednostavnijih pristupa, koji uprkos tome daje odlične rezultate.

---

### Naivni Bayes kao klasifikator sentimenta teksta

Sentiment analiza teksta predstavlja identifikaciju i ekstraktovanje subjektivnih informacija iz teksta. Najjednostavniji primer sentiment analize je procena da li je neki teskt napisan u pozitivnom ili negativnom kontekstu. Ovo je trenutno veoma aktuelna oblast istraživanja zbog eksplozije korišćenja socijalnih mreža - npr. nakon objavljivanja neke vesti ili statusa vezano za neki proizvod, veliki brendovi često žele da znaju kako su korisnici to prihvatili (bez "ručnog" analiziranja komentar po komentar).

Na ovim vežbama, koristićemo Naivni Bayesov klasifikator za klasifikaciju sentimenta korisničkih **realnih** recenzija filmova i knjiga, ali su ove recenzije skupljane na društvenim mrežama (što se oslikava malo "slobodnijim" rečnikom). Dataset za ovaj se nalazi u ```train.tsv```, gde prva kolona predstavlja sentiment recenzije (**pos** = pozitivna recenzija, **neg** = negativna recenzija), a druga kolona tekst same recenzije.

**Primeri pozitivnih recenzija:**
* Hey I loved The Da Vinci Code!..
* mission impossible 2 rocks!!....
* lol I love Harry Potter like a fat kid loves cake.

**Primeri negativnih recenzija:**
* Brokeback Mountain was depressing!
* I hate Harry Potter even more now. >:
* "Christmas Mission's Trip with our very own Tom Cruise: "" Mission Impossible is for stupid people "".."

Korišćenjem Naivnog Bayesa, problem klasifikator sentimenta teksta možemo definisati kao:

```
P(S) = P(recenzija je određenog sentimenta), P(s1) za pozitivnu, P(s2) za negativnu
P(T) = P(napisan određeni niz reči koji predstavlja recenziju)
P(T|S) = P(određen niz reči predstavlja recenziju određenog sentimenta)
P(S|T) = ovo je ono što treba izračunati -> za dati niz reči koji predstavlja recenziju izračunati verovatnoću da je ta recenzija određenog sentimenta

... naravno, isto ovako za negativne recenzije.
```

Obratite pažnju da je u prethodnim izrazima tekst (recenzija) predstavljan kao "niz reči", stoga ćemo primenjivati verziju Naivnog Bayesa prilagođenu ovom problemu.

$$
P(s_i|T) = \frac{P(T | s_i) \, P(s_i)}{P(T)}
$$

ako tekst T razložimo na reči $ t \in T $ od kojih se sastoji, problem transformišemo u:

$$
P(s_i|T) = \prod_{t \in T}\frac{P(t| s_i)}{P(t)} \, P(s_i)
$$

Pošto su $ \frac{P(t| s_i)}{P(t)} $ brojevi manji od 1, računanje više ovakvih brojeva može prouzrokovati problem numeričke prirode - *floating point undeflow*. Stoga ćemo iskoristiti jedan "trik" tako što ćemo izraz logaritmovati (i samim tim proizvod pretvoriti u sumu).

$$
ln(P(s_i|T)) = \sum_{t \in T}ln(\frac{P(t| s_i)}{P(t)}) \, + ln(P(s_i))
$$

Zatim, vratimo ovo sve u originalni oblik:

$$
P(s_i|T) = e^{\sum_{t \in T}ln(\frac{P(t| s_i)}{P(t)}) \, + ln(P(s_i))}
$$


**Važna napomena:** Postupak se naziva "naivni" jer se pretpostavlja da su stavke u formuli međusobno nezavisne. U našem slučaju, realno je očekivati da reči u tekstu nisu potpuno nezavisne, jer reči zajedno daju značenje celoj rečenici. Dakle, mi suštinski "kršimo" osnovnu pretpostavku ovog algoritma, ali videćete da će svejedno davati zadovoljavajuće rezultate.

---

Pošto ćemo raditi sa slobodnim tekstom (prirodnim jezikom) i rečima iz teksta, neophodno je uraditi nekoliko stvari kako bi se tekst uprostio i olakšao za dalju upotrebu. Neke od tehnika preprocesiranja teksta su:

* Izbacivanje tzv. **STOP** reči - to su reči koje se često pojavljuju i ne nose neko relevantno značenje za samu klasifikaciju teksta. Npr. za engleski jezik te reči su: ```a, an, the, and, of, for, ...```
* Izbacivanje znakova interpunkcije
* Stemizacija reči - svođenje reči na korenski oblik. Npr. ```running -> run, computers -> computer, glasses -> glass, ...```
* Svođenje svih reči na mala slova - lowercase
* itd.

Ovo preprocesiranje teksta često predstavlja veoma važan korak u implemetaciji dobrih klasifikatora, i značajno utiče na performanse klasifikatora.

---

### Zadaci

**TODO 1:** Učitati recenzije i sentimente u metodi ```load_data()```, iz datoteke ```data/train.tsv```. Rezultat metode treba da budu dve liste: ```texts``` i ```sentiments```.

**TODO 2:** Implementirati jednostavno preprocesiranje teksta (metoda ```preprocess(text)```):

* izbacivanje znakova interpunkcije
* svođenje celog teksta na mala slova

**TODO 3:** Implementirati tokenizaciju teksta na reči (metoda ```tokenize(text)```). Rezultat treba da bude lista reči koje se pojavljuju u datom tekstu.

**TODO 4:** Implementirati prebrojavanje reči u datum tekstu (metoda ```count_words(text)```). Rezultat treba da bude mapa, čiji ključevi su reči, a vrednosti broj ponavljanja te reči u datom tekstu.

**TODO 5:** Napuniti *bag-of-words* strukture. *Bag-of-words* je struktura koja za ceo korpus tekstova daje mapu, čiji ključevi su reči, a vrednosti broj ponavljanja te reči u celom korpusu tekstova.

**TODO 6:** Implementirati Naivni Bayesov klasifikator za sentiment teksta (recenzije). Rezultat treba da bude mapa verovatnoća koja opisuje u kojoj meri je dati tekst (recenzija) klasifikovan kao pozitivan i negativan.


```
P(bilo koja recenzija pozitivna) = ukupan broj pozitivnih recenzija / ukupan broj svih recenzija
P(bilo koja recenzija negativna) = ukupan broj pozitivnih negativna / ukupan broj svih recenzija

P(reč se pojavljuje u recenziji) = broj ponavljanja ove reči u celom korpusu / ukupan broj reči u celom korpusu

P(reč se pojavljuje u poz. recenziji) = (broj ponavljanja ove reči u poz. korpusu + 1) / (ukupan broj reči u poz. korpusu + ukupan broj reči u rečniku)
P(reč se pojavljuje u neg. recenziji) = (broj ponavljanja ove reči u neg. korpusu + 1) / (ukupan broj reči u neg. korpusu + ukupan broj reči u rečniku)
```


**TODO (domaći):** Implementirati jednostavnu stemizaciju reči i izbacivanje osnovnih STOP reči u koraku preprocesiranja teksta.

---


### Zaključak

* Naivni Bayesov klasifikator je izuzetno jednostavna probabilistička metoda
* Iako jednostavan, veoma efikasan - sve se svodi samo na računanje verovatnoća
* Iako se često krši osnovna pretpostavka postupka (da su promenljive međusobno nezavisne), i dalje se dobijaju dobri rezultati
* Naivni Bayes se ne koristi samo za klasifikaciju teksta, već naprotiv, i za mnoge druge potrebe: klasifikaciju (u opštem smislu), klasifikaciju slika, dijagnozu bolesti na osnovu simptoma, itd...


#### Prednosti

* Brzo "obučavanje" - jednom se prođe kroz podatke, tzv. *single-scan*
* Brza klasifikacija
* Nije osetljiv na šum i nerelevantne osobine
* Može da koristi i realne i diskretne podatke

#### Mane

* Pretpostavlja da su osobine podataka međusobno nezavisne (veoma često netačno)

---

**DODATNO:**

* Primeniti Naivni Bayes za implementaciju klasifikatora novinskih članaka na 20 grupa, data set dostupan na http://qwone.com/~jason/20Newsgroups/. (1 boda)